# Set up

In [1]:
# update path with data dir
import sys
sys.path.append('../../data/')

In [2]:
import warnings
# warnings.filterwarnings('ignore')

In [40]:
import pandas as pd
import patsy
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import PredefinedSplit, GridSearchCV, cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.backend import clear_session
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Getting data

In [4]:
file_name = '../csvs/element_gameweek_features_v03.csv'

In [5]:
element_gameweek_df = pd.read_csv(file_name)

In [6]:
current_event = 20

In [7]:
element_gameweek_df =\
element_gameweek_df[
    (element_gameweek_df['event'] <= current_event)
]

In [8]:
element_gameweek_df.fillna(3, inplace=True)

In [9]:
element_gameweek_df.head()

,safe_web_name,element,event,fixture,team,opposition_team,was_home,element_type,value,is_sunday,...,rolling_var_total_points_against_opposition_team_element_type,rolling_avg_total_points_home,rolling_var_total_points_home,rolling_avg_total_points_against_opposition_ground_element_type,rolling_var_total_points_against_opposition_ground_element_type,total_points,expected_avg_total_points_against_opposition_team,expected_var_total_points_against_opposition_team,expected_avg_total_points_at_ground,expected_var_total_points_at_ground
0,cech,1,1,1,1,13,1,1,50,1,...,3.0,3.0,3.0,3.0,3.0,3,3.000000,3.000000,3.000000,3.0
1,cech,1,2,14,1,6,0,1,50,0,...,3.0,3.0,3.0,3.0,3.0,3,0.681818,3.000000,3.000000,3.0
2,cech,1,3,21,1,19,1,1,50,0,...,8.0,3.0,3.0,3.0,3.0,3,3.750000,0.000000,3.000000,3.0
3,cech,1,4,33,1,5,0,1,50,1,...,11.0,3.0,3.0,6.0,3.0,1,4.493304,0.000000,4.901786,3.0
4,cech,1,5,46,1,15,0,1,50,0,...,49.0,3.0,0.0,2.0,0.0,2,3.663651,5.929855,1.332237,0.0


In [10]:
element_gameweek_df = element_gameweek_df[element_gameweek_df['rolling_avg_minutes_element_p3'] >= 60]

# Modelling data

## Splitting data

In [11]:
last_train_event = 15

In [12]:
test_fold = [-1 if i <= last_train_event else 0 for i in element_gameweek_df['event']]
ps = PredefinedSplit(test_fold)

In [13]:
for train_index, test_index in ps.split():
    element_gameweek_df_train, element_gameweek_df_test = \
    element_gameweek_df.copy().iloc[train_index], element_gameweek_df.copy().iloc[test_index]

## Building model

In [14]:
formula = \
'total_points ~ C(element) + C(element_type) + C(team) + C(opposition_team) + ' + \
' + '.join([i for i in element_gameweek_df.columns if i not in
 [
     'safe_web_name',
     'element',
     'event',
     'fixture',
     'team',
     'opposition_team',
     'element_type',
     'rolling_avg_attempted_passes_element',
     'rolling_var_total_points_against_opposition_team',
     'rolling_var_total_points_own_team',
     'rolling_var_total_points_opposition_team',
     'rolling_var_total_points_element_type',
     'rolling_var_total_points_against_opposition_team_element_type',
     'expected_var_total_points_against_opposition_team',
     'expected_var_total_points_at_ground',
     'rolling_avg_total_points_home',
     'rolling_var_total_points_home',
     'rolling_avg_total_points_against_opposition_ground_element_type',
     'rolling_var_total_points_against_opposition_ground_element_type',
     'total_points'
 ]])

In [15]:
print(formula)

total_points ~ C(element) + C(element_type) + C(team) + C(opposition_team) + was_home + value + is_sunday + is_weekday + is_late + is_early + rolling_avg_total_points_element + rolling_var_total_points_element + rolling_max_total_points_element + rolling_avg_minutes_element_p3 + rolling_avg_bps_element + rolling_avg_bonus_element + rolling_avg_goals_scored_element + rolling_avg_minutes_element + rolling_avg_clean_sheets_element + rolling_avg_completed_passes_element + rolling_avg_recoveries_element + rolling_avg_assists_element + rolling_avg_key_passes_element + rolling_avg_winning_goals_element + rolling_avg_tackled_element + rolling_avg_dribbles_element + rolling_avg_clearances_blocks_interceptions_element + rolling_avg_big_chances_created_element + rolling_avg_target_missed_element + rolling_avg_fouls_element + rolling_avg_tackles_element + rolling_avg_open_play_crosses_element + rolling_avg_offside_element + rolling_avg_big_chances_missed_element + rolling_avg_saves_element + rolli

In [16]:
y, X = patsy.dmatrices(formula, element_gameweek_df, return_type='matrix')

In [17]:
for train_index, test_index in ps.split():
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [18]:
n_features = X.shape[1]

## Selecting features

In [19]:
# estimator = GradientBoostingRegressor(n_estimators=120,
#                                       max_features=0.7,
#                                       min_samples_leaf=0.015,
#                                       learning_rate=0.03,
#                                       criterion='mse',
#                                       random_state=0)
# selector = RFECV(estimator, step=1,
#                  scoring='neg_mean_squared_error',
#                  cv=ps, n_jobs=1, verbose=1)
# selector.fit(X, y.ravel())

In [20]:
# features = patsy.dmatrices(formula, element_gameweek_df, return_type='dataframe')[1].columns
# selected_features = pd.Series(features)[selector.support_]
# list(selected_features)

In [21]:
# stark_X_sel = X[:,selector.support_]
# stark_X_train_sel = X_train[:,selector.support_]
# stark_X_test_sel = X_test[:,selector.support_]

In [22]:
X_sel = X
X_train_sel = X_train
X_test_sel = X_test

## Tuning model

### Helpers

In [73]:
def create_model(hidden_nodes,
                 hidden_layers,
                 dropout_rate,
                 weight_constraint
                ):
    model = Sequential()
    model.add(Dense(hidden_nodes,
                    activation='relu',
                    kernel_constraint=MaxNorm(weight_constraint)
                   ))
    
    
    for i in range(0, hidden_layers):
        model.add(Dropout(dropout_rate))
        model.add(Dense(hidden_nodes, activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
        
        

    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model

### GP tuner

In [98]:
space = [
    Real(30, 50, 'uniform', name='hidden_nodes'),
    Integer(0, 1, name='hidden_layers'),
    Real(20, 50, 'log-uniform', name='epochs'),
    Real(20, 150, 'log-uniform', name='batch_size'),
    Real(0.0, 0.9, 'uniform', name='dropout_rate'),
    Integer(2, 4, name='weight_constraint')
]

In [99]:
@use_named_args(space)
def objective(hidden_nodes,
              hidden_layers,
              epochs,
              batch_size,
              dropout_rate,
              weight_constraint
             ):
    
    model = create_model(
        hidden_nodes=int(hidden_nodes),
        hidden_layers=int(hidden_layers),
        dropout_rate=dropout_rate,
        weight_constraint=weight_constraint
    )
    
    model.fit(X_train_sel,
              y_train.ravel(),
              epochs=int(epochs),
              batch_size=int(batch_size))
    
    y_pred = model.predict(X_test_sel).flatten()
    test_loss = mean_squared_error(y_test, y_pred)

    del model
    clear_session()
    
    return test_loss

In [100]:
res_gp_arr = []
for i in range(1, 6):
    print('minimisation iteration', i)
    res_gp = gp_minimize(objective, space, n_calls=30, verbose=2)
    res_gp_arr.append(res_gp)

minimisation iteration 1
Iteration No: 1 started. Evaluating function at random point.
Epoch 1/20
2696/2696 [==============================] - 1s 265us/sample - loss: 18.7406
Epoch 2/20
2696/2696 [==============================] - 0s 79us/sample - loss: 8.6492
Epoch 3/20
2696/2696 [==============================] - 0s 80us/sample - loss: 8.4291
Epoch 4/20
2696/2696 [==============================] - 0s 74us/sample - loss: 8.3188
Epoch 5/20
2696/2696 [==============================] - 0s 50us/sample - loss: 8.2364
Epoch 6/20
2696/2696 [==============================] - 0s 47us/sample - loss: 8.1642
Epoch 7/20
2696/2696 [==============================] - 0s 46us/sample - loss: 8.1032
Epoch 8/20
2696/2696 [==============================] - 0s 48us/sample - loss: 7.9803
Epoch 9/20
2696/2696 [==============================] - 0s 47us/sample - loss: 7.9145
Epoch 10/20
2696/2696 [==============================] - 0s 45us/sample - loss: 7.8987
Epoch 11/20
2696/2696 [===========================

2696/2696 [==============================] - 0s 85us/sample - loss: 9.1894
Epoch 11/29
2696/2696 [==============================] - 0s 87us/sample - loss: 8.8057
Epoch 12/29
2696/2696 [==============================] - 0s 87us/sample - loss: 8.8295
Epoch 13/29
2696/2696 [==============================] - 0s 87us/sample - loss: 8.9892
Epoch 14/29
2696/2696 [==============================] - 0s 87us/sample - loss: 8.9981
Epoch 15/29
2696/2696 [==============================] - 0s 85us/sample - loss: 8.7292
Epoch 16/29
2696/2696 [==============================] - 0s 87us/sample - loss: 8.7366
Epoch 17/29
2696/2696 [==============================] - 0s 87us/sample - loss: 8.9128
Epoch 18/29
2696/2696 [==============================] - 0s 83us/sample - loss: 8.7847
Epoch 19/29
2696/2696 [==============================] - 0s 86us/sample - loss: 8.7025
Epoch 20/29
2696/2696 [==============================] - 0s 85us/sample - loss: 8.7800
Epoch 21/29
2696/2696 [==============================] 

2696/2696 [==============================] - 0s 54us/sample - loss: 9.0252
Epoch 34/38
2696/2696 [==============================] - 0s 55us/sample - loss: 8.8888
Epoch 35/38
2696/2696 [==============================] - 0s 57us/sample - loss: 8.9168
Epoch 36/38
2696/2696 [==============================] - 0s 55us/sample - loss: 8.9312
Epoch 37/38
2696/2696 [==============================] - 0s 56us/sample - loss: 8.9925
Epoch 38/38
2696/2696 [==============================] - 0s 57us/sample - loss: 8.8064
Iteration No: 6 ended. Evaluation done at random point.
Time taken: 6.9607
Function value obtained: 11.5586
Current minimum: 9.6058
Iteration No: 7 started. Evaluating function at random point.
Epoch 1/33
2696/2696 [==============================] - 0s 153us/sample - loss: 13.7584
Epoch 2/33
2696/2696 [==============================] - 0s 73us/sample - loss: 8.7263
Epoch 3/33
2696/2696 [==============================] - 0s 78us/sample - loss: 8.3685
Epoch 4/33
2696/2696 [==============

2696/2696 [==============================] - 0s 26us/sample - loss: 7.6935
Iteration No: 9 ended. Evaluation done at random point.
Time taken: 2.9318
Function value obtained: 9.6964
Current minimum: 9.5787
Iteration No: 10 started. Evaluating function at random point.
Epoch 1/41
2696/2696 [==============================] - 0s 100us/sample - loss: 22.6781
Epoch 2/41
2696/2696 [==============================] - 0s 17us/sample - loss: 10.8060
Epoch 3/41
2696/2696 [==============================] - 0s 17us/sample - loss: 9.7199
Epoch 4/41
2696/2696 [==============================] - 0s 16us/sample - loss: 9.0759
Epoch 5/41
2696/2696 [==============================] - 0s 16us/sample - loss: 8.7352
Epoch 6/41
2696/2696 [==============================] - 0s 17us/sample - loss: 8.5201
Epoch 7/41
2696/2696 [==============================] - 0s 16us/sample - loss: 8.4599
Epoch 8/41
2696/2696 [==============================] - 0s 16us/sample - loss: 8.3483
Epoch 9/41
2696/2696 [==================

Epoch 49/49
2696/2696 [==============================] - 0s 40us/sample - loss: 7.6381
Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 7.4080
Function value obtained: 9.6573
Current minimum: 9.5787
Iteration No: 12 started. Searching for the next optimal point.
Epoch 1/49
2696/2696 [==============================] - 0s 94us/sample - loss: 17.3471
Epoch 2/49
2696/2696 [==============================] - 0s 14us/sample - loss: 10.7246
Epoch 3/49
2696/2696 [==============================] - 0s 14us/sample - loss: 9.3145
Epoch 4/49
2696/2696 [==============================] - 0s 14us/sample - loss: 8.9298
Epoch 5/49
2696/2696 [==============================] - 0s 14us/sample - loss: 8.6817
Epoch 6/49
2696/2696 [==============================] - 0s 14us/sample - loss: 8.4617
Epoch 7/49
2696/2696 [==============================] - 0s 13us/sample - loss: 8.3695
Epoch 8/49
2696/2696 [==============================] - 0s 14us/sample - loss: 8.2479
Epoch 9/49
2696/

...

2696/2696 [==============================] - 0s 14us/sample - loss: 8.0268
Epoch 47/49
2696/2696 [==============================] - 0s 15us/sample - loss: 8.0277
Epoch 48/49
2696/2696 [==============================] - 0s 14us/sample - loss: 8.0078
Epoch 49/49
2696/2696 [==============================] - 0s 14us/sample - loss: 8.0141
Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 3.6551
Function value obtained: 9.6659
Current minimum: 9.5288
Iteration No: 21 started. Searching for the next optimal point.
Epoch 1/20
2696/2696 [==============================] - 0s 114us/sample - loss: 14.9699
Epoch 2/20
2696/2696 [==============================] - 0s 13us/sample - loss: 9.1654
Epoch 3/20
2696/2696 [==============================] - 0s 14us/sample - loss: 8.6427
Epoch 4/20
2696/2696 [==============================] - 0s 13us/sample - loss: 8.5350
Epoch 5/20
2696/2696 [==============================] - 0s 13us/sample - loss: 8.4476
Epoch 6/20
2696/2696 [===

/home/tom/footbot/venv/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Epoch 1/20
2696/2696 [==============================] - 0s 116us/sample - loss: 8.9219
Epoch 2/20
2696/2696 [==============================] - 0s 13us/sample - loss: 8.6632
Epoch 3/20
2696/2696 [==============================] - 0s 14us/sample - loss: 8.5669
Epoch 4/20
2696/2696 [==============================] - 0s 13us/sample - loss: 8.4490
Epoch 5/20
2696/2696 [==============================] - 0s 13us/sample - loss: 8.3869
Epoch 6/20
2696/2696 [==============================] - 0s 14us/sample - loss: 8.3341
Epoch 7/20
2696/2696 [==============================] - 0s 13us/sample - loss: 8.3361
Epoch 8/20
2696/2696 [==============================] - 0s 13us/sample - loss: 8.3031
Epoch 9/20
2696/2696 [==============================] - 0s 13us/sample - loss: 8.2396
Epoch 10/20
2696/2696 [==============================] - 0s 13us/sample - loss: 8.1585
Epoch 11/20
2696/2696 [==============================] - 0s 13us/sample - loss: 8.1087
Epoch 12/20
2696/2696 [============================

Epoch 1/49
2696/2696 [==============================] - 0s 140us/sample - loss: 12.0088
Epoch 2/49
2696/2696 [==============================] - 0s 66us/sample - loss: 8.5547
Epoch 3/49
2696/2696 [==============================] - 0s 66us/sample - loss: 8.4587
Epoch 4/49
2696/2696 [==============================] - 0s 66us/sample - loss: 8.3200
Epoch 5/49
2696/2696 [==============================] - 0s 67us/sample - loss: 8.1825
Epoch 6/49
2696/2696 [==============================] - 0s 66us/sample - loss: 8.0177
Epoch 7/49
2696/2696 [==============================] - 0s 68us/sample - loss: 8.1290
Epoch 8/49
2696/2696 [==============================] - 0s 66us/sample - loss: 8.0672
Epoch 9/49
2696/2696 [==============================] - 0s 66us/sample - loss: 8.0269
Epoch 10/49
2696/2696 [==============================] - 0s 66us/sample - loss: 7.9878
Epoch 11/49
2696/2696 [==============================] - 0s 66us/sample - loss: 8.2248
Epoch 12/49
2696/2696 [===========================

Epoch 44/49
2696/2696 [==============================] - 0s 45us/sample - loss: 7.6639
Epoch 45/49
2696/2696 [==============================] - 0s 45us/sample - loss: 7.8094
Epoch 46/49
2696/2696 [==============================] - 0s 46us/sample - loss: 7.7780
Epoch 47/49
2696/2696 [==============================] - 0s 45us/sample - loss: 7.6292
Epoch 48/49
2696/2696 [==============================] - 0s 45us/sample - loss: 7.6513
Epoch 49/49
2696/2696 [==============================] - 0s 47us/sample - loss: 7.6624
Iteration No: 27 ended. Search finished for the next optimal point.
Time taken: 8.2278
Function value obtained: 9.7734
Current minimum: 9.4418
Iteration No: 28 started. Searching for the next optimal point.
Epoch 1/20
2696/2696 [==============================] - 0s 120us/sample - loss: 13.4028
Epoch 2/20
2696/2696 [==============================] - 0s 14us/sample - loss: 10.5481
Epoch 3/20
2696/2696 [==============================] - 0s 14us/sample - loss: 9.3413
Epoch 4/20

In [101]:
res_gp_df = pd.DataFrame([i.x for i in res_gp_arr],
             columns=[
                 'hidden_nodes', 'hidden_layers', 'epochs', 'batch_size',
                 'dropout_rate', 'weight_constraint'
             ])

res_gp_df['MSE'] = [i.fun for i in res_gp_arr]

res_gp_df.sort_values('MSE')

,hidden_nodes,hidden_layers,epochs,batch_size,dropout_rate,weight_constraint,MSE
3,30.178032,1,20.000000,150.000000,0.01575,4,9.377819
2,34.551328,1,20.000000,116.849727,0.00000,3,9.393079
4,30.000000,1,20.000000,150.000000,0.00000,2,9.441787
0,30.000000,0,20.000000,150.000000,0.00000,4,9.443499
1,30.000000,0,36.738161,150.000000,0.00000,2,9.498944


### Grid search

In [84]:
model = KerasRegressor(build_fn=create_model)

In [114]:
hidden_nodes = [20, 25, 30, 35]
hidden_layers = [1]
epochs = [15, 20, 25]
batch_size = [100, 150]
dropout_rate = [0.01, 0.015, 0.02]
weight_constraint = [3, 4]


parameters = {
    'hidden_nodes': hidden_nodes,
    'hidden_layers': hidden_layers,
    'epochs': epochs,
    'batch_size': batch_size,
    'dropout_rate': dropout_rate,
    'weight_constraint': weight_constraint
}

In [115]:
gs_model = GridSearchCV(
    model,
    parameters,
    scoring=['neg_mean_absolute_error', 'neg_mean_squared_error'],
    refit=False,
    cv=ps,
    error_score=100, n_jobs=1, verbose=2)
gs_model.fit(X_sel, y.ravel())

Fitting 1 folds for each of 144 candidates, totalling 144 fits
[CV] batch_size=100, dropout_rate=0.01, epochs=15, hidden_layers=1, hidden_nodes=20, weight_constraint=3 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/15
2696/2696 [==============================] - 5s 2ms/sample - loss: 25.6371
Epoch 2/15
2696/2696 [==============================] - 0s 24us/sample - loss: 10.2682
Epoch 3/15
2696/2696 [==============================] - 0s 23us/sample - loss: 9.8719
Epoch 4/15
2696/2696 [==============================] - 0s 23us/sample - loss: 9.3597
Epoch 5/15
2696/2696 [==============================] - 0s 24us/sample - loss: 9.3075
Epoch 6/15
2696/2696 [==============================] - 0s 24us/sample - loss: 9.2318
Epoch 7/15
2696/2696 [==============================] - 0s 25us/sample - loss: 8.9056
Epoch 8/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.8603
Epoch 9/15
2696/2696 [==============================] - 0s 26us/sample - loss: 9.0297
Epoch 10/15
2696/2696 [==============================] - 0s 27us/sample - loss: 8.7067
Epoch 11/15
2696/2696 [==============================] - 0s 26us/sample - loss: 8.6547
Epoch 12/15
2696/2696 [============================

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.9s remaining:    0.0s


Epoch 1/15
2696/2696 [==============================] - 5s 2ms/sample - loss: 17.6540
Epoch 2/15
2696/2696 [==============================] - 0s 24us/sample - loss: 9.2164
Epoch 3/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.8120
Epoch 4/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.6965
Epoch 5/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.6403
Epoch 6/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.6896
Epoch 7/15
2696/2696 [==============================] - 0s 24us/sample - loss: 8.5649
Epoch 8/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.4960
Epoch 9/15
2696/2696 [==============================] - 0s 24us/sample - loss: 8.4109
Epoch 10/15
2696/2696 [==============================] - 0s 24us/sample - loss: 8.4096
Epoch 11/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.3668
Epoch 12/15
2696/2696 [=============================

...

2696/2696 [==============================] - 0s 36us/sample - loss: 8.4702
Epoch 10/25
2696/2696 [==============================] - 0s 32us/sample - loss: 8.4363
Epoch 11/25
2696/2696 [==============================] - 0s 32us/sample - loss: 8.4209
Epoch 12/25
2696/2696 [==============================] - 0s 32us/sample - loss: 8.3493
Epoch 13/25
2696/2696 [==============================] - 0s 32us/sample - loss: 8.3450
Epoch 14/25
2696/2696 [==============================] - 0s 32us/sample - loss: 8.2127
Epoch 15/25
2696/2696 [==============================] - 0s 36us/sample - loss: 8.2469
Epoch 16/25
2696/2696 [==============================] - 0s 41us/sample - loss: 8.1536
Epoch 17/25
2696/2696 [==============================] - 0s 40us/sample - loss: 8.2420
Epoch 18/25
2696/2696 [==============================] - 0s 38us/sample - loss: 8.1960
Epoch 19/25
2696/2696 [==============================] - 0s 34us/sample - loss: 8.1613
Epoch 20/25
2696/2696 [==============================] 

2696/2696 [==============================] - 0s 34us/sample - loss: 8.2051
Epoch 21/25
2696/2696 [==============================] - 0s 35us/sample - loss: 8.1738
Epoch 22/25
2696/2696 [==============================] - 0s 35us/sample - loss: 8.1031
Epoch 23/25
2696/2696 [==============================] - 0s 34us/sample - loss: 8.1024
Epoch 24/25
2696/2696 [==============================] - 0s 36us/sample - loss: 8.1360
Epoch 25/25
2696/2696 [==============================] - 0s 34us/sample - loss: 8.1342
[CV]  batch_size=150, dropout_rate=0.02, epochs=25, hidden_layers=1, hidden_nodes=25, weight_constraint=4, total=  54.7s
[CV] batch_size=150, dropout_rate=0.02, epochs=25, hidden_layers=1, hidden_nodes=30, weight_constraint=3 
Epoch 1/25
2696/2696 [==============================] - 36s 13ms/sample - loss: 15.7577
Epoch 2/25
2696/2696 [==============================] - 0s 30us/sample - loss: 9.1965
Epoch 3/25
2696/2696 [==============================] - 0s 40us/sample - loss: 9.0992
Epo

2696/2696 [==============================] - 0s 35us/sample - loss: 9.3311
Epoch 5/25
2696/2696 [==============================] - 0s 35us/sample - loss: 8.9631
Epoch 6/25
2696/2696 [==============================] - 0s 35us/sample - loss: 8.8835
Epoch 7/25
2696/2696 [==============================] - 0s 37us/sample - loss: 8.9222
Epoch 8/25
2696/2696 [==============================] - 0s 37us/sample - loss: 8.7451
Epoch 9/25
2696/2696 [==============================] - 0s 35us/sample - loss: 8.6001
Epoch 10/25
2696/2696 [==============================] - 0s 37us/sample - loss: 8.5924
Epoch 11/25
2696/2696 [==============================] - 0s 43us/sample - loss: 8.6047
Epoch 12/25
2696/2696 [==============================] - 0s 45us/sample - loss: 8.3171
Epoch 13/25
2696/2696 [==============================] - 0s 48us/sample - loss: 8.4383
Epoch 14/25
2696/2696 [==============================] - 0s 39us/sample - loss: 8.2950
Epoch 15/25
2696/2696 [==============================] - 0s 

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 72.4min finished


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             error_score=100,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f712028d550>,
             iid='warn', n_jobs=1,
             param_grid={'batch_size': [100, 150],
                         'dropout_rate': [0.01, 0.015, 0.02],
                         'epochs': [15, 20, 25], 'hidden_layers': [1],
                         'hidden_nodes': [20, 25, 30, 35],
                         'weight_constraint': [3, 4]},
             pre_dispatch='2*n_jobs', refit=False, return_train_score=False,
             scoring=['neg_mean_absolute_error', 'neg_mean_squared_error'],
             verbose=2)

In [116]:
gs_model_df = pd.DataFrame(gs_model.cv_results_)[[
    'param_hidden_nodes', 'param_hidden_layers', 'param_epochs',
    'param_batch_size', 'param_dropout_rate', 'param_weight_constraint',
    'split0_test_neg_mean_absolute_error',
    'split0_test_neg_mean_squared_error'
]]

gs_model_df

,param_hidden_nodes,param_hidden_layers,param_epochs,param_batch_size,param_dropout_rate,param_weight_constraint,split0_test_neg_mean_absolute_error,split0_test_neg_mean_squared_error
0,20,1,15,100,0.01,3,-2.124033,-9.619317
1,20,1,15,100,0.01,4,-2.110486,-9.736834
2,25,1,15,100,0.01,3,-2.088165,-9.728637
3,25,1,15,100,0.01,4,-2.186907,-9.500431
4,30,1,15,100,0.01,3,-2.242478,-9.546927
5,30,1,15,100,0.01,4,-2.266384,-9.568992
6,35,1,15,100,0.01,3,-2.219422,-9.550918
7,35,1,15,100,0.01,4,-2.190175,-9.518123
8,20,1,20,100,0.01,3,-2.386283,-9.546420
9,20,1,20,100,0.01,4,-2.206465,-9.371396


In [117]:
gs_model_df[
    gs_model_df['split0_test_neg_mean_squared_error'] == \
    gs_model_df['split0_test_neg_mean_squared_error'].max()]

,param_hidden_nodes,param_hidden_layers,param_epochs,param_batch_size,param_dropout_rate,param_weight_constraint,split0_test_neg_mean_absolute_error,split0_test_neg_mean_squared_error
9,20,1,20,100,0.01,4,-2.206465,-9.371396


# Evaluating model

In [ ]:
model = create_model(hidden_nodes,
                 hidden_layers,
                 dropout_rate,
                 weight_constraint
                )

model.fit(X_train_sel,
          y_train.ravel(),
          epochs=00,
          batch_size=00)

In [ ]:
# get predictions for test set
y_pred = model.predict(X_test_sel).flatten()

# calculate cross entropy loss for test set
test_loss = (
    mean_absolute_error(y_test, y_pred),
    mean_squared_error(y_test, y_pred),
    r2_score(y_test, y_pred)
)
test_loss

In [ ]:
pd.Series(y_pred).describe()

In [ ]:
# refit model on all data
model_full = GradientBoostingRegressor(n_estimators=120,
                                            max_features=0.61,
                                            min_samples_leaf=0.015,
                                            learning_rate=0.035,
                                            criterion='mse',
                                            random_state=0)
model_full.fit(X_sel, y.ravel())

In [ ]:
# get predictions for whole set
y_pred_full = model_full.predict(X_sel).flatten()

# calculate cross entropy loss for test set
test_loss_full = (
    mean_absolute_error(y, y_pred_full),
    mean_squared_error(y, y_pred_full),
    r2_score(y, y_pred_full)
)
test_loss_full

In [ ]:
pd.Series(y_pred_full).describe()

In [ ]:
abs(pd.Series(y.ravel()) - pd.Series(y_pred_full)).describe()

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(pd.Series(y_pred_full), bins=range(-5,20), alpha=0.75, density=True, label='predicted')
plt.hist(pd.Series(y.ravel()), bins=range(-5,20), alpha=0.75, density=True, label='observed')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(pd.Series(y.ravel()), pd.Series(y_pred_full), alpha=0.1)
plt.plot(np.linspace(-5, 25), np.linspace(-5, 25))
plt.ylabel('Predicted total points')
plt.xlabel('Observed total points')
plt.ylim((0, 7))
plt.show()

## Summary

In [ ]:
pd.DataFrame([
    ('Neural network', test_loss[0], test_loss[1], test_loss[2]),
], columns=['model', 'MAE', 'MSE', 'r2'])

In [ ]:
pd.DataFrame([
    ('Neural network', test_loss_full[0], test_loss_full[1], test_loss_full[2]),
], columns=['model', 'MAE', 'MSE', 'r2'])